In [1]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

import pandas as pd

load_dotenv()  # loads from .env by default

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)




In [3]:
# read the data from the CSV file 100per categoriy
import pandas as pd

df = pd.read_csv("100percategoryjun23.csv")


In [4]:
# from sqlalchemy import create_engine
# import pandas as pd
#
# # Create engine with only client-side encoding
# engine = create_engine(connection_string)
#
# # Clean function that ensures LATIN1 compatibility
# def clean_for_latin1(df):
#     df_clean = df.copy()
#     for col in df_clean.select_dtypes(include=['object']).columns:
#         df_clean[col] = df_clean[col].astype(str).apply(
#             lambda x: x.encode('latin1', errors='replace').decode('latin1')
#         )
#     return df_clean
#
# # Clean and export the data
# df_clean = clean_for_latin1(df)
# df_clean.to_sql("multiplecategories", engine, index=False, if_exists="replace")

549

In [7]:
# # from csv
# all_articles_by_category = pd.read_csv("all_articles_by_category.csv")
#
# all_articles_cleaned = clean_for_latin1(all_articles_by_category)
# all_articles_cleaned.to_sql("all_articles_by_category", engine, index=False, if_exists="replace")

695

In [18]:
import pandas as pd

# Read both tables from the database
full_query = """SELECT m.*, a.title, c.root, c.stratum
                FROM multiplecategories m
                         LEFT JOIN all_articles_by_category c ON m.article_id = c.article_id
                         LEFT JOIN articles a ON m.article_id = a.article_id;"""

# execute the query and load the data into a DataFrame

merged = pd.read_sql(full_query, engine)




In [19]:
merged.head()

,snapshot_ts,rev_id,timestamp,user,is_bot,content,article_id,title,root,stratum
0,2023-06-30 00:00:00+00:00,1160763005,2023-06-18 16:30:41+00:00,2601:483:C301:7360:BC05:287F:176:F15F,False,{{Use dmy dates|date=July 2021}}\n{{Short desc...,20951,List of empires,History,History
1,2023-07-31 00:00:00+00:00,1166013433,2023-07-18 21:17:52+00:00,TompaDompa,False,{{Use dmy dates|date=July 2021}}\n{{Short desc...,20951,List of empires,History,History
2,2023-08-31 00:00:00+00:00,1171485250,2023-08-21 11:13:21+00:00,79.41.96.200,False,{{Use dmy dates|date=July 2021}}\n{{Short desc...,20951,List of empires,History,History
3,2023-09-30 00:00:00+00:00,1177319025,2023-09-27 04:45:15+00:00,187.254.98.237,False,{{Use dmy dates|date=July 2021}}\n{{Short desc...,20951,List of empires,History,History
4,2023-10-31 00:00:00+00:00,1181899435,2023-10-25 22:06:39+00:00,Idulatria,False,{{Use dmy dates|date=July 2021}}\n{{Short desc...,20951,List of empires,History,History


In [20]:
# write merged data to a new CSV file
merged.to_csv("100percategoryjun23_with_categories.csv", index=False)

# Add baseline and delta columns


In [2]:
# read the merged data from the CSV file
after_delta = pd.read_csv("after_spacy_parsed100percat_with_categories_june23_with_delta.csv")

In [3]:
update_df = after_delta[["rev_id", "p_t", "lexical_spike_delta"]].copy()


In [5]:
update_df.to_sql("temp_revision_updates", engine, index=False, if_exists="replace")

832

In [7]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("""
        UPDATE multiplecategories AS m
        SET
            p_t = t.p_t,
            lexical_spike_delta = t.lexical_spike_delta
        FROM temp_revision_updates AS t
        WHERE m.rev_id = t.rev_id;
    """))
